In [2]:
!pip install -q crewai 'crewai[tools]' langchain_groq --progress-bar off
!pip install gradio

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.17.1 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.
tensorflow-metadata 1.13.1 requires protobuf<5,>=3.20.3, but you have protobuf 5.29.2 which is incompatible.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 93.1 MB/s eta 0:00:00
  Attempting uninstall: markupsaf

In [3]:
from google.colab import userdata
import os
from crewai import Agent, Task, Crew, LLM
from typing import Dict, Optional
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [4]:
try:
    groq_api_key = userdata.get('groq_api')
    serper_api_key = userdata.get('serper_api')

    if not groq_api_key or not serper_api_key:
        raise ValueError("API keys not found")

    os.environ["GROQ_API_KEY"] = groq_api_key
    os.environ["SERPER_API_KEY"] = serper_api_key
except Exception as e:
    logger.error(f"Error setting up API keys: {str(e)}")
    raise

In [5]:
groq_llm = LLM(
    model="groq/llama3-8b-8192",
    temperature=0.3,
    max_tokens=4096,
    api_key=groq_api_key,
)

In [6]:
class ContentAgent:
    def __init__(self, role: str, goal: str, backstory: str) -> None:
        self.role = role
        self.goal = goal
        self.backstory = backstory

    def create_agent(self, llm: Optional[LLM] = None) -> Agent:
        return Agent(
            role=self.role,
            goal=self.goal,
            backstory=self.backstory,
            allow_delegation=False,
            verbose=True,
            llm=llm or groq_llm
        )

In [7]:
planner = ContentAgent(
    role="Content Strategy Director",
    goal="Plan engaging, factually accurate, and SEO-optimized content on {topic}",
    backstory="""You are a seasoned Content Strategy Director with over 15 years of experience in digital content planning.
    Your expertise lies in identifying trending topics, understanding audience demographics, and creating data-driven content strategies.
    You've successfully led content teams at major digital publications and have a deep understanding of SEO best practices.
    You're known for your ability to spot emerging trends before they go mainstream and for creating content plans that consistently
    drive high engagement rates. You have a master's degree in Digital Communications and regularly speak at industry conferences
    about content strategy and audience engagement. Your previous work has resulted in significant traffic increases for multiple
    Fortune 500 companies."""
).create_agent()

writer = ContentAgent(
    role="Senior Content Writer",
    goal="Write insightful and factually accurate opinion pieces about {topic}",
    backstory="""You are an accomplished Senior Content Writer with a unique background combining journalism and technical expertise.
    You've spent 10 years as a technology journalist for leading publications, covering emerging technologies and their impact on society.
    Your writing style is known for making complex topics accessible while maintaining technical accuracy. You've won several
    awards for your in-depth analysis pieces and have authored two books on digital transformation. You hold a degree in
    Computer Science and a master's in Journalism, giving you a rare combination of technical knowledge and storytelling ability.
    You're particularly skilled at conducting research, interviewing experts, and weaving compelling narratives that engage
    both technical and non-technical audiences. Your articles consistently rank in the top 1% for reader engagement and social shares."""
).create_agent()

editor = ContentAgent(
    role="Executive Editor",
    goal="Polish and perfect content while maintaining editorial standards and brand voice",
    backstory="""You are an Executive Editor with over 20 years of experience in digital publishing and content optimization.
    Your career began at traditional print magazines, where you developed a keen eye for detail and storytelling. You successfully
    led the digital transformation of several major publications, pioneering new content formats and editorial workflows.
    You've managed editorial teams across multiple continents and have developed style guides used by hundreds of writers.
    Your expertise includes SEO optimization, content governance, and maintaining brand consistency across various platforms.
    You've served on the board of several journalism organizations and have been recognized with industry awards for editorial
    excellence. You're known for your ability to maintain high editorial standards while meeting aggressive publishing schedules.
    You have a PhD in Communications and regularly contribute to academic journals on digital content strategy."""
).create_agent()

In [8]:
def create_task_templates() -> Dict[str, Task]:
    return {
        "plan": Task(
            description="""
            1. Conduct comprehensive research on {topic}:
               - Analyze current market trends and industry developments
               - Identify key stakeholders and their perspectives
               - Research competitor content and identify gaps
               - Gather relevant statistics and case studies

            2. Develop audience persona:
               - Define demographic and psychographic characteristics
               - Map out user journey and pain points
               - Identify information needs and preferences
               - Determine optimal content format and tone

            3. Create detailed content strategy:
               - Outline main themes and subtopics
               - Develop key messaging points
               - Plan content structure and flow
               - Identify supporting materials needed

            4. SEO and distribution strategy:
               - Research and prioritize target keywords
               - Plan internal and external linking strategy
               - Identify distribution channels
               - Set measurable success metrics
            """,
            expected_output="""A comprehensive content strategy document including:
            - Detailed audience analysis
            - Content outline with supporting research
            - SEO strategy and keywords
            - Distribution plan and success metrics""",
            agent=planner
        ),
        "write": Task(
            description="""
            1. Content Creation:
               - Write engaging introduction that hooks readers
               - Develop comprehensive main sections
               - Create compelling calls-to-action
               - Include relevant examples and case studies

            2. Technical Requirements:
               - Implement SEO best practices
               - Format content for web readability
               - Include proper citations and sources
               - Optimize for target keywords

            3. Quality Standards:
               - Ensure factual accuracy
               - Maintain consistent tone and style
               - Follow brand guidelines
               - Check for plagiarism

            4. Enhancement:
               - Add relevant statistics and data
               - Include expert quotes where appropriate
               - Suggest visual content opportunities
               - Optimize for social sharing
            """,
            expected_output="A polished, publication-ready blog post in markdown format with proper formatting, citations, and optimized for both readers and search engines.",
            agent=writer
        ),
        "edit": Task(
            description="""
            1. Content Review:
               - Evaluate overall structure and flow
               - Verify factual accuracy and citations
               - Check alignment with content strategy
               - Assess tone and style consistency

            2. Technical Edit:
               - Correct grammar and punctuation
               - Optimize sentence structure
               - Verify keyword usage and density
               - Check formatting and layout

            3. Enhancement:
               - Strengthen weak sections
               - Add clarifying information
               - Optimize headlines and subheadings
               - Improve transitions

            4. Final Polish:
               - Verify brand voice consistency
               - Check for SEO best practices
               - Ensure mobile-friendly formatting
               - Prepare for publication
            """,
            expected_output="A fully edited, polished, and publication-ready blog post that meets all quality standards and is optimized for both readers and search engines.",
            agent=editor
        )
    }

In [9]:
def initialize_crew(topic: str) -> Crew:
    try:
        tasks = create_task_templates()
        crew = Crew(
            agents=[planner, writer, editor],
            tasks=[tasks["plan"], tasks["write"], tasks["edit"]],
            verbose=True
        )
        return crew
    except Exception as e:
        logger.error(f"Error initializing crew: {str(e)}")
        raise

def generate_blog_post(topic: str) -> str:
    try:
        crew = initialize_crew(topic)
        result = crew.kickoff(inputs={"topic": topic})
        return result
    except Exception as e:
        logger.error(f"Error generating blog post: {str(e)}")
        raise

In [10]:
if __name__ == "__main__":
    try:
        result = generate_blog_post("AI in human resources")
        print(result)
    except Exception as e:
        logger.error(f"Error in main execution: {str(e)}")

# Agent: Content Strategy Director
## Task: 
            1. Conduct comprehensive research on AI in human resources:
               - Analyze current market trends and industry developments
               - Identify key stakeholders and their perspectives
               - Research competitor content and identify gaps
               - Gather relevant statistics and case studies

            2. Develop audience persona:
               - Define demographic and psychographic characteristics
               - Map out user journey and pain points
               - Identify information needs and preferences
               - Determine optimal content format and tone

            3. Create detailed content strategy:
               - Outline main themes and subtopics
               - Develop key messaging points
               - Plan content structure and flow
               - Identify supporting materials needed

            4. SEO and distribution strategy:
               - Research and prioriti

In [11]:
import gradio as gr

In [13]:
def gradio_interface(topic: str) -> str:
    try:
        return generate_blog_post(topic)
    except Exception as e:
        return f"Error: {str(e)}"

interface = gr.Interface(
    fn=gradio_interface,
    inputs="text",
    outputs="text",
    title="AI Blog Post Generator",
    description="Enter a topic to generate a comprehensive blog post using the AI-powered content generation system."
)

# Cell 10: Launch Gradio Interface
if __name__ == "__main__":
    interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5271dbd5b6de633d75.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
